시소러스 : 체계화된 사전. 상위 하위개념을 트리로 체계화한것 - 사람이 수작업으로 만든 결과물 예: 워드넷
단점 : 시대변화에 대응이 어려움(수작업), 사람이 쓰는 비용이 큼, 단어의 미묘한 차이를 표현할 수 없다

희소표현 : 원핫벡터로 나타낸 값
단점 :
1. 벡터의 크기가 너무 크다(10000개의 단어면 10000개의 차원이 나타남)
2. 벡터가 단어의 의미를 포함하지 않는다+ 순서를 고려하지 않는다

분포가설 : 단어의 의미는 그 자체보다 단어가 사용된 맥락이 의미를 형성한다

동시발생행렬 : 주어진 모든 단어들에 대해 동시발생하는 단어를 표에 정리한 것
(항상 대칭인 행렬), i say he is 라는 문장이 있을 때 say라는 단어를 선택했을 때 양 옆의 단어인 i 와 he만 보는 것

코사인유사도 : 가리키는 방향이 같으면 1, 반대방향이면0
x * y / llxll * llyll

llxll, llyll에서 ll는 노름(norm)을 나타낼 때 사용. 의미 : 길이 또는 크기가 부여된 벡터 공간

In [1]:
#argsort 설명 = 오름차순 순서대로 인덱싱값 추출하는 것
import numpy as np
x=np.array([100,-20, 2])
print(x)
print(x.argsort())
print((-x).argsort()) #내림차순으로 정렬하고 싶을 때

[100 -20   2]
[1 2 0]
[0 2 1]


In [2]:
import sys
sys.path.append('..')
import os
#from common.np import *    pip install common을 해도 진행되지 않음.

In [3]:
#전처리 함수
def preprocess(text): #텍스트 삽입
    text = text.lower() # 모든 문자 소문자로
    text= text.replace('.', ' .') # .만 있는 것을 한칸 띄우고 .을 사용 => 밑에서 쓸 스플릿으로 나누기 위함
    words = text.split(' ') #빈 칸을 기준으로 나누는데, 위에서 사용한 전처리로인해 온점이 단어와 띄어쓰여진 채로 인식됨
    
    word_to_id = {} # you:0 ......
    id_to_word = {} # 0:you ......
    for word in words:
        if word not in word_to_id: #처음은 당연히 없음. 중복되지 않는한 이 구문 실행
            new_id = len(word_to_id) #길이지정. 처음은 아무것도 없으므로 new_id=0
            word_to_id[word] = new_id # 처음// word_to_id[you]=0 => word_to_id딕셔너리에서 you가 키값, 0이 value값
            id_to_word[new_id] = word # 처음// id_to_word[0]=you => id_to_word 딕셔너리의 키값이 0 밸류값이 you
            
    corpus = np.array([word_to_id[w] for w in words]) #삽입 문장을 word_to_id에 넣어서 숫자 value값을 출력 => 형식은 array
    
    return corpus, word_to_id, id_to_word

In [4]:
#동시발생행렬 생성
def create_co_matrix(corpus, vocab_size, window_size=1):
    #corpus =말뭉치, vocab_size= 어휘 수, window_size = 윈도우 크기
    #윈도우 크기가 1이면 타깃 단어 기준 좌우 한단어씩만 맥락에 포함
    
    corpus_size = len(corpus) #문장의 길이로 지정
    co_matrix= np.zeros((vocab_size, vocab_size), dtype=np.int32) #vocab_size*vocab_size의 행렬 생성
    
    for idx, word_id in enumerate(corpus): #enumerate를 사용해서 idx=>0..., word_id=>you...... 형태로 for 문에 넣음
        for i in range(1, window_size+1):
            left_idx = idx - i   #처음 left_idx는 0-1 => -1 이므로 위쪽if는 실행되지 않음// 두번째부터 실행
            right_idx = idx + i  #제일 마지막 단어가 올 경우+1을 하게됨=> 마지막 단어는 두번째 if문을 통과하지 못함
            
            if left_idx >= 0: #위쪽 if               #단어 인덱싱한것-i가 0보다 큰 경우
                left_word_id = corpus[left_idx]       #left_word_id = corpus[0] [1] ..... 
                co_matrix[word_id, left_word_id] += 1  # word_id * left_word_id 행렬을 만들고 +1을 함 
                                                       #행렬(1,0) 자리에 1을 추가하라
                
            if right_idx < corpus_size: #아래쪽 if     #단어 인덱싱한것+i가 삽입단어보다 작을경우
                right_word_id = corpus[right_idx]       #right_word_id = corpus[1] [2] .... 가 오고 마지막 단어가 타겟 단어가 될 경우 이 부분은 실행되지 않음
                co_matrix[word_id, right_word_id] +=1    #word_id * right_word_id 행렬을 만들고 +1을 함
                
    return co_matrix

In [5]:
#코사인 유사도 산출
def cos_similarity(x,y,eps=1e-8):  #eps를 넣는 이유는 같이 들어가는값이 0일경우 오류발생
    #x,y 모두 벡터
    nx = x/(np.sqrt(np.sum(x**2))+eps) #벡터x를 x의 크기로 나눠주기// nx=normalize x
    ny = y/(np.sqrt(np.sum(y**2))+eps) # y제곱의 합에 루트를 씌운 것이 스칼라값을 구하는 방법
    return np.dot(nx,ny) #np.dot은 element wise곱을 할 때 사용

In [6]:
#유사 단어 검색
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # query =쿼리(텍스트), word_to_id = 단어:인덱스의 딕셔너리, id_to_word = 인덱스:단어의 딕셔너리
    #word_matrix = 단어벡터를 정리한 행렬 = 동시발생행렬(각 행에 벡터가 저장되어 있다고 가정)
    #param top : 상위 몇 개까지 출력할 것인지?
    
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)  #딕셔너리에 단어가 없을 경우 오류메시지 출력
        
    print('\n[query] ' + query)  # \n을 사용해서 한줄 밑으로
    query_id = word_to_id[query]  #키값이 문장인 딕셔너리에 키값 query를 넣은값 = value(=숫자)를 query_id로 지정
    print(query_id)
    query_vec = word_matrix[query_id] #동시발생행렬 0번째 행???????????????????
    print(query_vec)                  #query_id에 숫자가 들어가는 것은 알겠는데 행 전체를 지정하는 것인지??
    
    #코사인 유사도 계산
    vocab_size = len(id_to_word) #포함한 단어의 수
    
    similarity = np.zeros(vocab_size) #어휘수가 7개면 0으로 채워진 7차원 벡터를 만듬 => 7*7행렬

    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec) #7차원벡터 similarity[i번째] = 
        
    #코사인 유사도를 기준으로 내림차순 출력
    count = 0
    for i in (-1* similarity).argsort(): # 5번셀 설명
        if id_to_word[i] == query:     #i번째 단어와 query가 같을때
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:     #5이상일경우 끝내기
            return

In [7]:
import sys
sys.path.append('..')


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
0
[0 1 0 0 0 0 0]
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
